In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_api'.")

# Configurer le module avec la clé API
genai.configure(api_key=api_key)

/home/maximeadmin/anaconda3/envs/mon_env_ia/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Importation des bibliothèques nécessaires
import os
import re
import json
import time
from pathlib import Path
from dotenv import load_dotenv
import google.generativeai as genai

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()
api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_gemini_api'.")

# Configurer l'API Gemini
genai.configure(api_key=api_key)

def extract_uuid(content):
    """Extraire l'UUID du contenu LaTeX."""
    uuid_match = re.search(r'\\uuid\{([^}]+)\}', content)
    return uuid_match.group(1) if uuid_match else None

def create_prompt(latex_content):
    """Créer le prompt pour l'API Gemini avec une attention particulière aux compétences."""
    return f"""Voici un exercice de mathématiques au format LaTeX. Analyse-le et génère les métadonnées suivantes:

1. Compétences mathématiques requises pour résoudre l'exercice (5 maximum)
   - Exprime chaque compétence sous forme d'un verbe d'action suivi d'un complément précis
   - Utilise des verbes comme: calculer, démontrer, dériver, résoudre, appliquer, interpréter, modéliser
   - Fais correspondre la compétence à une action concrète que l'étudiant doit maîtriser
   - Liste les compétences par ordre d'importance (la plus cruciale en premier)
   - Exemple: "calculer une dérivée partielle" plutôt que "dérivée partielle"

2. Niveau de difficulté (facile, intermédiaire, avancé)

3. Mots-clés pertinents pour la recherche (8 maximum)

4. Concepts théoriques fondamentaux mobilisés (3-5)

5. Prérequis nécessaires pour aborder l'exercice

6. Type d'exercice (Exercice d'application directe, Problème à étapes, problème ouvert, Démonstration, Modélisation, Investigation)

7. Temps estimé pour la résolution

Réponds uniquement en format JSON structuré avec ces 7 clés exactes:
{{
  "competences": ["compétence 1", "compétence 2", ...],
  "niveau_difficulte": "niveau",
  "mots_cles": ["mot-clé 1", "mot-clé 2", ...],
  "concepts_fondamentaux": ["concept 1", "concept 2", ...],
  "prerequis": ["prérequis 1", "prérequis 2", ...],
  "type_exercice": "type",
  "temps_estime": "temps"
}}

Voici l'exercice:
{latex_content}"""

def extract_json_from_response(text_response, debug=False):
    """Extraire le JSON de la réponse de l'API."""
    if debug:
        print("\n=== RÉPONSE BRUTE DE L'API ===")
        print(text_response)
        print("==============================\n")
    
    # Tenter différentes méthodes d'extraction
    try:
        # Méthode 1: JSON direct
        return json.loads(text_response)
    except json.JSONDecodeError:
        # Méthode 2: Bloc JSON markdown
        json_match = re.search(r'```json\n([\s\S]*?)\n```', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except:
                pass
        
        # Méthode 3: Bloc code générique
        json_match = re.search(r'```\n([\s\S]*?)\n```', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except:
                pass
        
        # Méthode 4: Expression régulière pour trouver tout ce qui ressemble à un JSON
        json_match = re.search(r'\{[\s\S]*\}', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(0))
            except:
                pass
        
        # Si toutes les méthodes échouent
        if debug:
            print("ERREUR: Impossible d'extraire un JSON valide de la réponse.")
        raise ValueError("Format de réponse non reconnu ou JSON invalide")

def analyze_latex_file(file_path, output_dir="output", model_name="gemini-2.0-flash", force_reprocess=False, debug=False):
    """Analyser un fichier LaTeX et générer les métadonnées via l'API Gemini."""
    try:
        # Lire le contenu du fichier
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extraire l'UUID
        uuid = extract_uuid(content)
        if not uuid:
            print(f"UUID non trouvé dans le fichier {file_path}")
            return None
        
        # Vérifier si le fichier a déjà été traité
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f"{uuid}.json")
        if os.path.exists(output_file) and not force_reprocess:
            print(f"Fichier {uuid} déjà traité, ignoré.")
            with open(output_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        
        # Créer le prompt
        prompt = create_prompt(content)
        
        if debug:
            print("\n=== PROMPT ENVOYÉ À L'API ===")
            print(prompt)
            print("============================\n")
        
        # Appeler l'API Gemini
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            text_response = response.text
        except Exception as e:
            print(f"ERREUR lors de l'appel à l'API Gemini: {e}")
            return None
        
        # Extraire le JSON de la réponse
        try:
            json_data = extract_json_from_response(text_response, debug)
            
            # Vérifier si les données sont vides ou nulles
            if not json_data:
                print("AVERTISSEMENT: Les données JSON extraites sont vides.")
                if debug:
                    print("JSON extrait:", json_data)
                return None
            
            # Créer l'objet de résultat final avec vérification de l'existence des clés
            metadata = {
                "competences": json_data.get("competences") or [],
                "niveau_difficulte": json_data.get("niveau_difficulte") or "",
                "mots_cles": json_data.get("mots_cles") or [],
                "concepts_fondamentaux": json_data.get("concepts_fondamentaux") or [],
                "prerequis": json_data.get("prerequis") or [],
                "type_exercice": json_data.get("type_exercice") or "",
                "temps_estime": json_data.get("temps_estime") or ""
            }
                        
            result = {uuid: metadata}
            
            # Écrire le résultat dans un fichier
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            
            return result
        except Exception as e:
            print(f"Erreur lors de l'extraction des métadonnées pour {uuid}: {e}")
            if debug:
                print(f"Réponse brute: {text_response}")
            return None
    except Exception as e:
        print(f"Erreur lors de l'analyse du fichier {file_path}: {e}")
        return None

def process_latex_files(source_dir="src/latex/amscc", output_dir="output", model_name="gemini-2.0-flash", 
                       max_files=None, api_delay=1.0, force_reprocess=False, debug=False):
    """Traiter tous les fichiers LaTeX dans le répertoire source."""
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtenir tous les fichiers .tex dans le répertoire source
    files = [str(f) for f in Path(source_dir).glob('**/*.tex')]
    
    # Limiter le nombre de fichiers si demandé
    if max_files:
        files = files[:max_files]
        print(f"Mode test: limité à {max_files} fichiers")
    
    print(f"Traitement de {len(files)} fichiers LaTeX...")
    
    # Traiter chaque fichier
    results = {}
    success_count = 0
    
    for i, file in enumerate(files):
        print(f"\n[{i+1}/{len(files)}] Analyse de {os.path.basename(file)}...")
        result = analyze_latex_file(file, output_dir, model_name, force_reprocess, debug)
        
        if result:
            results.update(result)
            success_count += 1
        
        # Pause entre les appels API (sauf pour le dernier fichier)
        if i < len(files) - 1:
            print(f"Pause de {api_delay} secondes...")
            time.sleep(api_delay)
    
    # Écrire tous les résultats dans un seul fichier
    with open(os.path.join(output_dir, 'all_results.json'), 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"Traitement terminé. {success_count}/{len(files)} fichiers analysés avec succès.")
    return results

In [ ]:
# Exemple d'utilisation (à exécuter dans une cellule séparée)
results = process_latex_files(
    source_dir="../src/latex/amscc",
    output_dir="../metadata/amscc-v2",  # Utiliser un répertoire différent pour les nouvelles métadonnées
    force_reprocess=False,               # Forcer le retraitement pour générer de nouvelles métadonnées
    api_delay=5                       # Pause entre les appels API
)